In [1]:
import numpy as np
import math
import os
from scipy.spatial.transform import Rotation as R
import quaternion
import cv2
import matplotlib.pyplot as plt
import scipy.ndimage
import torch
import torch.nn.functional as F
import tensorflow as tf

2022-02-25 18:01:24.797528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/shantanu.singh/miniconda3/envs/py37/lib
2022-02-25 18:01:24.797568: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# !ls /scratch/shantanu/results_23022022/occant_rgb/dump/0/Cantwell/front/bev_pred
# !ls /scratch/shantanu/gibson4/new/Cantwell/0/front/pose

In [107]:
# Beta : momentum for moving average update
beta = 0.99##
threshold_explored = 0.5 ##
threshold_entropy = 0.5 ##
EPS_MAPPER = 1e-6
s = 0.05
LOCAL_MAP_SIZE = 128##
OUTMAP_SIZE = 1024##

In [38]:
def aggregate(m,p_reg):
    """
    Inputs:
        m - ( 2, M, M) - global map
        p_reg - ( 2, M, M) - registered egomap
    """
#     explored_mask = (p_reg[:, 1] > threshold_explored).float()
    explored_mask = (p_reg[1] > threshold_explored)
    '''
    Looks like they want sone kind of count on 
    number of observations for that cell
    '''
    
    log_p_reg = np.log(p_reg + EPS_MAPPER)
    log_1_p_reg = np.log(1 - p_reg + EPS_MAPPER)
#     log_p_reg = torch.log(p_reg + EPS_MAPPER)
#     log_1_p_reg = torch.log(1 - p_reg + EPS_MAPPER)
    entropy = -p_reg * log_p_reg - (1 - p_reg) * log_1_p_reg
#     print("Entropy shape : ",entropy.shape)
#     entropy_mask = (entropy.mean(dim=1) < threshold_entropy).float()
    '''
    Entroypy mask mean along ???
    '''
    entropy_mask = (entropy.mean(axis=0) < threshold_entropy)
    
    explored_mask = explored_mask * entropy_mask
#     unfilled_mask = (m[:, 1] == 0).float()
    unfilled_mask = (m[1] == 0)
    
    m_updated = m
    # For regions that are unfilled, write as it is
    mask = unfilled_mask * explored_mask
#     mask = mask.unsqueeze(1)
    m_updated = m_updated * (1 - mask) + p_reg * mask
    # For regions that are filled, do a moving average
    mask = (1 - unfilled_mask) * explored_mask
#     mask = mask.unsqueeze(1)
    p_reg_ma = (p_reg * (1 - beta) + m_updated * beta) * mask
    m_updated = m_updated * (1 - mask) + p_reg_ma * mask
    
    return m_updated

In [116]:
def spatial_transform_map(p, x, invert=True, mode="linear"):
    """
    Inputs:
        p     - ( f, H, W) Tensor
        x     - (3) Tensor (x, y, theta) transforms to perform
    Outputs:
        p_trans - (bs, f, H, W) Tensor
    Conventions:
        Shift in X is rightward, and shift in Y is downward. Rotation is clockwise.
    Note: These denote transforms in an agent's position. Not the image directly.
    For example, if an agent is moving upward, then the map will be moving downward.
    To disable this behavior, set invert=False.
    """
#     device = p.device
#     H, W = p.shape[2:]
    H, W = p.shape[1:]
    
#     trans_x = x[:, 0]
#     trans_y = x[:, 1]
    trans_x = x[0]
    trans_y = x[1]
    # Convert translations to -1.0 to 1.0 range
    Hby2 = (H - 1) / 2 if H % 2 == 1 else H / 2
    Wby2 = (W - 1) / 2 if W % 2 == 1 else W / 2

    trans_x = trans_x / Wby2
    trans_y = trans_y / Hby2
#     rot_t = x[:, 2]
    rot_t = x[2]

#     sin_t = torch.sin(rot_t)
#     cos_t = torch.cos(rot_t)
    sin_t = np.sin(rot_t)
    cos_t = np.cos(rot_t)
    # This R convention means Y axis is downwards.
#     A = np.zeros((p.size(0), 3, 3)).to(device)
#     A = np.zeros((p.shape[0], 3, 3))
    A = np.zeros((3, 3))
    
#     A[:, 0, 0] = cos_t
#     A[:, 0, 1] = -sin_t
#     A[:, 1, 0] = sin_t
#     A[:, 1, 1] = cos_t
#     A[:, 0, 2] = trans_x
#     A[:, 1, 2] = trans_y
#     A[:, 2, 2] = 1
    A[0, 0] = cos_t
    A[0, 1] = -sin_t
    A[1, 0] = sin_t
    A[1, 1] = cos_t
    A[0, 2] = trans_x
    A[1, 2] = trans_y
    A[2, 2] = 1
    # Since this is a source to target mapping, and F.affine_grid expects
    # target to source mapping, we have to invert this for normal behavior.
#     Ainv = torch.inverse(A)
    Ainv = np.linalg.inv(A)
    print(A)
    print(np.round(Ainv,2))

    # If target to source mapping is required, invert is enabled and we invert
    # it again.
    if invert:
#         Ainv = torch.inverse(Ainv)
        Ainv = np.linalg.inv(Ainv)
        
#     Ainv = Ainv[:, :2]
#     Ainv = Ainv[:2]
#     print(Ainv)
#     print(p.shape,Ainv.shape)
#     p_trans = cv2.warpAffine(p[0],Ainv[:-1,:],(1024,1024))
    p_trans = scipy.ndimage.affine_transform(p,Ainv)
#     print(p_trans.shape)
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(p[0])
    plt.subplot(1,2,2)
    plt.imshow(p_trans[0])
#     grid = F.affine_grid(Ainv, p.size())
#     print(np.unique(p))
#     plt.
#     p_trans = F.grid_sample(p, grid, mode=mode)
    '''
    Initially it was mode="bilinear " but in scipy.interpolate.griddata ,
    no such mode is present. So i set the default to "linear"
    '''
#     p_trans = scipy.interpolate.griddata(p, grid, mode=mode)

    return p_trans

def spatial_transform(p, dx, invert=False):
    # Convert dx to map image coordinate system with X as rightward and Y as downward
#     dx_map = torch.stack(
#         [(dx[:, 1] / s), -(dx[:, 0] / s), dx[:, 2]], dim=1
#     )  # anti-clockwise rotation
#     dx_map = np.stack(
#         [(dx[:, 1] / s), -(dx[:, 0] / s), dx[:, 2]], axis=1
#     )
    dx_map = np.stack(
        [(dx[1] / s), -(dx[0] / s), dx[2]]
    )
#     print(dx_map)
    p_trans = spatial_transform_map(p, dx_map, invert=invert)

    return p_trans

In [40]:
def register_map(m,p,x):
    """
    Given the locally computed map, register it to the global map based
    on the current position.
    Inputs:
    m - ( F, M, M) global map
    p - ( F, V, V) local map
    x - ( 3) in global coordinates
    """
    V = LOCAL_MAP_SIZE
    M = m.shape[2]
    Vby2 = (V - 1) // 2 if V % 2 == 1 else V // 2
    Mby2 = (M - 1) // 2 if M % 2 == 1 else M // 2
    # The agent stands at the bottom-center of the egomap and looks upward
    left_h_pad = Mby2 - V + 1
    right_h_pad = M - V - left_h_pad
    left_w_pad = Mby2 - Vby2
    right_w_pad = M - V - left_w_pad
    # Add zero padding to p so that it matches size of global map
#     print(p.shape)
    p_pad = np.zeros((p.shape[0],M,M))
    p_pad[:,left_h_pad:left_h_pad+V,left_w_pad:left_w_pad+V] = p
#     print(p_pad.shape)

#     p_pad = F.pad(
#             p, (left_w_pad, right_w_pad, left_h_pad, right_h_pad), "constant", value=0
#         )
#     p_pad = np.pad(
#     p, np.array([(left_w_pad, right_w_pad, left_h_pad, right_h_pad),(left_w_pad, right_w_pad, left_h_pad, right_h_pad)]).T, "constant")
    # Register the local map
    p_reg = spatial_transform(p_pad, x)
    # Aggregate
    m_updated = aggregate(m, p_reg)

    return m_updated

    

In [115]:
LOCAL_MAP_DIR  = "/scratch/shantanu/results_23022022/occant_rgb/dump/0/Cantwell/front/bev_pred"
POSE_DIR  = "/scratch/shantanu/gibson4/new/Cantwell/0/front/pose"

def register_all_observations():
#     m = np.zeros((1,2,OUTMAP_SIZE,OUTMAP_SIZE))
    m = np.zeros((2,OUTMAP_SIZE,OUTMAP_SIZE))
#     m = tf.convert_to_tensor(m)
    local_maps = os.listdir(LOCAL_MAP_DIR)
    for local_map in local_maps[300:301]:
        scan = local_map.split('.')[0]
        pose = np.load(POSE_DIR+'/'+scan+'.npy',allow_pickle=True).item()
        px,pz = pose['position'][0],pose['position'][2]
        quat = pose['rotation']
        yaw = quaternion.as_euler_angles(quat)[2]
        angles = quaternion.as_euler_angles(quat)
        
        print(angles)
#         x = np.array([[px,pz,yaw]])
        x = np.array([px,pz,yaw])
        
#         x = tf.convert_to_tensor(x)
#         print(x)
        p = cv2.imread(LOCAL_MAP_DIR+'/'+local_map,-1)
        unknown_p = (1-(p==0))
        p_reg = np.zeros((2,LOCAL_MAP_SIZE,LOCAL_MAP_SIZE))
        p_reg[0,:] = p
        p_reg[1,:] = unknown_p
#         plt.imshow(unknown_p)
#         print(p_reg.shape)
#         print(np.unique(p_reg))
#         p_reg = np.array([np.moveaxis(np.array(p_reg),-1,0)])
#         p_reg = np.moveaxis(np.array(p_reg),-1,0)
#         p_reg = tf.convert_to_tensor(p_reg)
#         print(p_reg)
        m = register_map(m,p_reg,x)
    return m
m = register_all_observations()
# plt.imshow(m[1])

[3.14159265 3.09285051 3.14159265]
